In this notebook, I'll be exploring the data related to other services, categorizing them, summarizing findings and saving the final results into csv files.

In [32]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import data:
df = pd.read_csv('original_other_services_data.csv')

In [3]:
df.describe(include = 'all')

,participant_ID,other_services_housing_utilities,other_services_computers_internet,other_services_tax_clinic_financial_services,other_services_legal,other_services_navigation,other_services_employment_income_support,other_services_education,other_services_childcare,other_services_none,other_services_prefer_not_to_answer,other_services_other
count,4054,771,838,541,402,347,675,513,479,1365,259,92
unique,4054,2,1,1,1,1,1,1,1,1,1,78
top,ns 757,housing / utilities supports,computers or internet,income tax clinics / financial literacy services,legal services,one on one service navigation,employment/income support,education services,childcare,none of the above,prefer not to answer,clothing
freq,1,716,838,541,402,347,675,513,479,1365,259,6


We can see that there are multiple values in the "housing utilities" and "other" columns, let's check them out:

In [4]:
df.other_services_housing_utilities.unique()

array([nan, 'housing / utilities supports',
       'housing  / utilities supports'], dtype=object)

In [5]:
to_fix = df[df.other_services_housing_utilities.isna() == False]

In [6]:
df.loc[to_fix.index,'other_services_housing_utilities'] = 'housing / utilities supports'

In [7]:
df.loc[to_fix.index].other_services_housing_utilities.value_counts()

other_services_housing_utilities
housing / utilities supports    771
Name: count, dtype: int64

In [8]:
df.other_services_other.unique()

array([nan, 'i donâ€™t think i use other services', 'medical checkup',
       'seasonal dinner for the clients are each others support systems',
       'i', 'food', "i didn't know other services were available",
       'no service', 'ontario', 'didnt know there were other options',
       'ontario works', 'debt help',
       'i did not know about these services through the food bank.',
       'community/social belonging', 'education services', 'library',
       'medical checkup and drugs', 'healthcare programs', 'none',
       'i need housing and legal services', 'none apply',
       'clothing and bedding',
       'the programs they offer are close to useless',
       'clothing offered at bronson centre', 'clothing', 'nothing',
       "i didn't no they had anything else",
       'was not aware other services are available', 'rent', 'disability',
       'bbq and special events', 'social interaction, and social events.',
       "i didn't know these were availible",
       'meal services 

In [9]:
df.columns

Index(['participant_ID', 'other_services_housing_utilities',
       'other_services_computers_internet',
       'other_services_tax_clinic_financial_services', 'other_services_legal',
       'other_services_navigation', 'other_services_employment_income_support',
       'other_services_education', 'other_services_childcare',
       'other_services_none', 'other_services_prefer_not_to_answer',
       'other_services_other'],
      dtype='object')

In [10]:
# Define keywords for each other services option
keywords = {
    'housing': ['housing', 'rent'],
    'computers_internet': ['computer'],
    'tax_clinic_financial_services': ['income tax', 'debt help', 'financial support'],
    'legal': ['legal services', 'advice'],
    'navigation': ['advocacy support', 'navigation'],
    'employment and income support': ['employment support', 'social services', 'worker', 'ontario'],
    'education': ['education', 'library', 'homework club'],
    'childcare': [ 'childcare'],
    'no other': ['nothing', "unaware", 'useless', 'uhh', 'food',  'none',  "no", 'only', 'hi', "i don't use other services", "i don't understand the question",],
    'prefer not to answer': ['prefer not to answer', 'program'],
    'children care supplies': ['diaper', 'baby milk', 'after school snack'],
    'socializing' : ['social support', 'community', 'belonging', 'bbq', 'socialize', 'events','activities', 'dinner'],
    'clothing' : ['cloth'],
    'laundry' : ['laundry'],
    'medical' : ['healthcare', 'medic', 'vaccination', 'disability', 'health'],
}

In [11]:
# I'll create columns for the new options:
df[['other_services_children_care_supplies', 'other_services_community_support', 'other_services_clothing','other_services_laundry','other_services_medical_services','other_services_no_other_services']] = None

In [12]:
# Function to classify answers based on keywords
def classify_service(answer, keywords):
        
    if isinstance(answer, str):
        answer_lower = answer.lower()
        matched_options = []
        
        if answer_lower == 'i':
            matched_options.append('none required')
        
        elif answer_lower == 'yes':
            matched_options.append('prefer not to answer')
        
        else:
            for option, keywords in keywords.items():
                if any(keyword in answer_lower for keyword in keywords):
                    if option not in matched_options:
                        matched_options.append(option)

        if len(matched_options) > 1:
            if 'prefer not to answer' in matched_options:
                matched_options.remove('prefer not to answer')
            elif 'none' in matched_options:
                matched_options.remove('none')
            
            
        return '; '.join(matched_options) if matched_options else answer
    
    return answer # Default to ''answer' if no match found


In [13]:
# Apply the function to map each answer to provided options
df['mapped_service'] = df['other_services_other'].apply(lambda x: classify_service(x, keywords))

In [14]:
df.mapped_service.value_counts()

mapped_service
no other                            37
medical                             10
no other; clothing                   9
socializing                          8
employment and income support        5
tax_clinic_financial_services        3
education                            3
computers_internet                   2
children care supplies               2
prefer not to answer                 2
socializing; medical                 2
laundry                              1
housing                              1
housing; legal                       1
navigation                           1
clothing                             1
no other; children care supplies     1
none required                        1
no other; medical                    1
legal                                1
Name: count, dtype: int64

In [15]:
df[df.mapped_service.str.endswith('none') == True].other_services_other

Series([], Name: other_services_other, dtype: object)

In [16]:
df.iloc[4000]

participant_ID                                                                            ns 4006
other_services_housing_utilities                                                              NaN
other_services_computers_internet                                                             NaN
other_services_tax_clinic_financial_services                                                  NaN
other_services_legal                                                                          NaN
other_services_navigation                                                                     NaN
other_services_employment_income_support                                                      NaN
other_services_education                                                                      NaN
other_services_childcare                                                                      NaN
other_services_none                                                                           NaN
other_services_prefe

In [17]:
#Now we can drop the  other column:
df.drop('other_services_other', axis = 1, inplace = True)

In [18]:
# create a function to consolidate the data:
def consolidate_row(row):
    """ 
    Consolidate answers from a DataFrame row, excluding the first and last columns.

    This function processes a row from a DataFrame to:
    1. Create a list of consolidated answers from specified columns, excluding the "participant_ID" (first column) and the last column.
       - Only non-null and non-empty values are included in the list.
    2. Count the number of answers provided.
    3. If the consolidated list contains "prefer not to answer", the list is updated to contain only this value.
    4. Return the consolidated answers as a single string and the count of answers.

    Parameters:
    row (pd.Series): A row from a DataFrame containing answers to be consolidated.

    Returns:
    tuple: A tuple containing:
           - A string of consolidated answers, joined by '; '.
           - An integer count of the number of answers provided.
    """
        
    consolidated = []
    
    # Loop through the specified columns excluding the "participant_ID":
    for value in row.iloc[1:]:
        if pd.notna(value)  and value != '':
            values = value.split(';') 
            for value in values:
                if value not in consolidated:
                    consolidated.append(value)
        
    
    consolidated_answer = ';'.join(consolidated)
    
    # finding out how many answers were provided:
    number_of_answers = len(consolidated_answer.split(';'))
    
    # ensuring we don't get any '' in the final result:
    if consolidated_answer == '':
        consolidated_answer = 'prefer not to answer'
        number_of_answers = 0
        
    # updating the records for "prefer not to answer":
    if 'prefer not to answer' in consolidated:
        consolidated_answer = 'prefer not to answer'        
    
    return consolidated_answer , number_of_answers

In [19]:
# Apply the consolidate_row function to each row and create new columns:
df[['other_services_consolidated', 'other_services_number_of_answers']] = df.apply(
        lambda row: pd.Series(consolidate_row(row)), axis=1 )

In [20]:
# Since all mapped modes are in the consolidated column, i'll drop the mapped column:
df.drop('mapped_service', axis = 1, inplace = True)

In [21]:
# overview:
df.describe(include = 'all')

,participant_ID,other_services_housing_utilities,other_services_computers_internet,other_services_tax_clinic_financial_services,other_services_legal,other_services_navigation,other_services_employment_income_support,other_services_education,other_services_childcare,other_services_none,other_services_prefer_not_to_answer,other_services_children_care_supplies,other_services_community_support,other_services_clothing,other_services_laundry,other_services_medical_services,other_services_no_other_services,other_services_consolidated,other_services_number_of_answers
count,4054,771,838,541,402,347,675,513,479,1365,259,0,0,0,0,0,0,4054,4054.000000
unique,4054,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,247,NaN
top,ns 757,housing / utilities supports,computers or internet,income tax clinics / financial literacy services,legal services,one on one service navigation,employment/income support,education services,childcare,none of the above,prefer not to answer,NaN,NaN,NaN,NaN,NaN,NaN,none of the above,NaN
freq,1,771,838,541,402,347,675,513,479,1365,259,NaN,NaN,NaN,NaN,NaN,NaN,1322,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.552787
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.066527
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000


In [22]:
# first, I'll set all values in the desired columns to 0
df.iloc[:, 1:-2] = 0

In [23]:
# Now, I am going to replace the values in each column: 1 is indicated in the column that matches any of the consoldiated value:
for x in df.index:
    values = df.other_services_consolidated.iloc[x].split(';')
    
    dictionary_words = {'housing': df.columns[1],
                     'computer' : df.columns[2],
                     'tax' : df.columns[3],
                     'legal' : df.columns[4],
                     'navigation' : df.columns[5],
                     'employment' : df.columns[6],
                     'education': df.columns[7],
                     'childcare' : df.columns[8],
                     'none of the above' : df.columns[9],
                     'prefer' : df.columns[10],
                     'children' : df.columns[11],
                     'socializing' : df.columns[12],
                     'clothing' : df.columns[13],
                    'laundry' : df.columns[14],
                    'medical' : df.columns[15],
                    'no other': df.columns[16]
                     }
    
    if 'prefer not to answer' in values:
        df.iloc[x, 10] = 1
            
    else:
        for value in values:
            for key in dictionary_words.keys():
                if key in value:
                    column = dictionary_words[key]
                    df.loc[x, [column]] = 1
                
                

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4054 entries, 0 to 4053
Data columns (total 19 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   participant_ID                                4054 non-null   object
 1   other_services_housing_utilities              4054 non-null   object
 2   other_services_computers_internet             4054 non-null   object
 3   other_services_tax_clinic_financial_services  4054 non-null   object
 4   other_services_legal                          4054 non-null   object
 5   other_services_navigation                     4054 non-null   object
 6   other_services_employment_income_support      4054 non-null   object
 7   other_services_education                      4054 non-null   object
 8   other_services_childcare                      4054 non-null   object
 9   other_services_none                           4054 non-null   object
 10  

In [25]:
df.describe(include = 'all')

,participant_ID,other_services_housing_utilities,other_services_computers_internet,other_services_tax_clinic_financial_services,other_services_legal,other_services_navigation,other_services_employment_income_support,other_services_education,other_services_childcare,other_services_none,other_services_prefer_not_to_answer,other_services_children_care_supplies,other_services_community_support,other_services_clothing,other_services_laundry,other_services_medical_services,other_services_no_other_services,other_services_consolidated,other_services_number_of_answers
count,4054,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054.0,4054,4054.000000
unique,4054,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,247,NaN
top,ns 757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,none of the above,NaN
freq,1,3288.0,3220.0,3513.0,3653.0,3710.0,3375.0,3540.0,3576.0,2697.0,3754.0,4051.0,4044.0,4044.0,4053.0,4041.0,4007.0,1322,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.552787
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.066527
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000


In [26]:
df.isna().sum()

participant_ID                                  0
other_services_housing_utilities                0
other_services_computers_internet               0
other_services_tax_clinic_financial_services    0
other_services_legal                            0
other_services_navigation                       0
other_services_employment_income_support        0
other_services_education                        0
other_services_childcare                        0
other_services_none                             0
other_services_prefer_not_to_answer             0
other_services_children_care_supplies           0
other_services_community_support                0
other_services_clothing                         0
other_services_laundry                          0
other_services_medical_services                 0
other_services_no_other_services                0
other_services_consolidated                     0
other_services_number_of_answers                0
dtype: int64

In [27]:
# I will create a summary where # of disabilities is categorized into more generic bins ('none', '1-2', '3+')
bins = {
    'prefer not to answer': df.other_services_consolidated == 'prefer not to answer',
    'none of the above' : df.other_services_none == 1,
    'no other service': df.other_services_no_other_services == 1,
    '1': (df.other_services_number_of_answers == 1),
    '2':  (df.other_services_number_of_answers == 2),
    '3+': df.other_services_number_of_answers >= 3,
}

In [28]:
# Create a new column in the DataFrame
df['services_bin'] = np.select(list(bins.values()),list(bins.keys()), default='unknown')

In [29]:
# now, i'll save the dataframe:
df.to_csv('NS_other_services_mapped_consolidated_binned.csv', index = False)

In [30]:
overall_summary = df.iloc[:, -2:].groupby('services_bin').count()

In [31]:
overall_summary =overall_summary.sort_values(by ='total', ascending = False)

KeyError: 'total'

In [ ]:
overall_summary.columns = ['total']

In [ ]:
# plotting the overall_summary results:
plt.figure(figsize=(12, 8))
sns.barplot(
    data= overall_summary,
    x ='services_bin',
    y ='total',
    errorbar=None
)

#plt.xticks(rotation=90)
plt.xlabel('Number of Services')
plt.ylabel('Total')
plt.title('Total Count per Number of Services')


# Adjust layout to make room for the legend
plt.tight_layout()

# Save the figure
plt.savefig('other_services_number_summary.png', dpi=300, bbox_inches='tight')

# show figure
plt.show()

In [ ]:
# I'll save the above summary:
overall_summary.to_csv('binned_services.csv')

In [ ]:
summary = {}
for column in df.columns[1:-3]:
    service = column.split('_')
    service = ' '.join(service[2:])
    
    if service == 'none':
        service = 'none of the above'
        
    summary.update({service: df[column].sum(axis = 0)})

In [ ]:
# create a dataframe from the above summary dictionary:
services_summary = pd.DataFrame({'service' : summary.keys(),
             'total' : summary.values()})

In [ ]:
services_summary = services_summary.sort_values(by = 'total', ascending = False)

In [ ]:
services_summary

In [ ]:
# save the summary dataframe:
services_summary.to_csv('other_services_summary.csv', index = False)

In [ ]:
# plotting the summary results:

plt.figure(figsize=(12, 8))
sns.barplot(
    data= services_summary,
    x ='total',
    y ='service',
    errorbar=None
)

plt.ylabel('Service')
plt.xlabel('Total')
plt.title('Total Count per Service')


# Adjust layout to make room for the legend
plt.tight_layout()

# Save the figure
plt.savefig('other_services_summary.png', dpi=300, bbox_inches='tight')

# show figure
plt.show()